# REST API
<img src='pics/0.png' />

# Serialization and Deserialization

<img src='pics/chicken-egg.png' />

## serialization

In [ ]:
object = dict(int_list=[1, 2, 3],
 
              text='string',
 
              number=3.44,
 
              boolean=True,
 
              none=None)

print(simple)

In [ ]:
{'none': None, 'int_list': [1, 2, 3], 'text': 'string', 'boolean': True, 'number': 3.44}

In [ ]:
print(json.dumps(object, indent=4))

In [ ]:
{
 
    "text": "string",
 
    "none": null,
 
    "boolean": true,
 
    "number": 3.44,
 
    "int_list": [
 
        1,
 
        2,
 
        3
 
    ]
 
}

### Deserialization

In [ ]:
serialized=json.dumps(object, indent=4)
print(serialized)

In [ ]:
{
    "none": null,
    "int_list": [
        1,
        2,
        3
    ],
    "text": "string",
    "boolean": true,
    "number": 3.44
}

In [ ]:
json.loads(serialized)

In [ ]:
{'text': 'string', 'int_list': [1, 2, 3], 'none': None, 'boolean': True, 'number': 3.44}

# Declaring Serializers

Serializers allow complex data such as querysets and model instances to be converted to native Python datatypes that can then be easily rendered into JSON, XML or other content types. 

 Serializers also provide deserialization, allowing parsed data to be converted back into complex types, after first validating the incoming data.
 
 The serializers in REST framework work very similarly to Django's Form and ModelForm classes. We provide a Serializer class which gives you a powerful, generic way to control the output of your responses, as well as a ModelSerializer class which provides a useful shortcut for creating serializers that deal with model instances and querysets.



In [ ]:
from datetime import datetime

class Comment(object):
    def __init__(self, email, content, created=None):
        self.email = email
        self.content = content
        self.created = created or datetime.now()

comment = Comment(email='leila@example.com', content='foo bar')

In [ ]:
from rest_framework import serializers

class CommentSerializer(serializers.Serializer):
    email = serializers.EmailField()
    content = serializers.CharField(max_length=200)
    created = serializers.DateTimeField()

# Serializing objects

At this point we've translated the model instance into Python native datatypes.

In [ ]:
serializer = CommentSerializer(comment)
serializer.data

{'email': 'leila@example.com', 'content': 'foo bar', 'created': '2016-01-27T15:17:10.375877'}

To finalise the serialization process we render the data into json

In [ ]:
from rest_framework.renderers import JSONRenderer

json = JSONRenderer().render(serializer.data)
json

b'{"email":"leila@example.com","content":"foo bar","created":"2016-01-27T15:17:10.375877"}'

# Deserializing objects

First we parse a stream into Python native datatypes...

In [ ]:
from django.utils.six import BytesIO
from rest_framework.parsers import JSONParser

stream = BytesIO(json)
data = JSONParser().parse(stream)

...then we restore those native datatypes into a dictionary of validated data.

In [ ]:
serializer = CommentSerializer(data=data)
serializer.is_valid()

True

In [ ]:
serializer.validated_data

{'content': 'foo bar', 'email': 'leila@example.com', 'created': datetime.datetime(2012, 08, 22, 16, 20, 09, 822243)}

# complete object instances based on the validated data

In [ ]:
class CommentSerializer(serializers.Serializer):
    email = serializers.EmailField()
    content = serializers.CharField(max_length=200)
    created = serializers.DateTimeField()

    def create(self, validated_data):
        return Comment(**validated_data)

    def update(self, instance, validated_data):
        instance.email = validated_data.get('email', instance.email)
        instance.content = validated_data.get('content', instance.content)
        instance.created = validated_data.get('created', instance.created)
        return instance

# object instances correspond to Django models

In [ ]:
def create(self, validated_data):
    return Comment.objects.create(**validated_data)

def update(self, instance, validated_data):
    instance.email = validated_data.get('email', instance.email)
    instance.content = validated_data.get('content', instance.content)
    instance.created = validated_data.get('created', instance.created)
    instance.save()
    return instance

# create a new instance, or update an existing instance

In [ ]:
# .save() will create a new instance.
serializer = CommentSerializer(data=data)
serializer.save()

In [ ]:
# .save() will update the existing `comment` instance.
serializer = CommentSerializer(comment, data=data)
serializer.save()

# Overriding .save() directly.

In some cases the .create() and .update() method names may not be meaningful. For example, in a contact form we may not be creating new instances, but instead sending an email or other message.

In these cases you might instead choose to override .save() directly, as being more readable and meaningful.

In [ ]:
class ContactForm(serializers.Serializer):
    email = serializers.EmailField()
    message = serializers.CharField()

    def save(self):
        email = self.validated_data['email']
        message = self.validated_data['message']
        send_email(from=email, message=message)

# Validation

In [ ]:
serializer = CommentSerializer(data={'email': 'foobar', 'content': 'baz'})
serializer.is_valid()

False

In [ ]:
serializer.errors

{'email': [u'Enter a valid e-mail address.'], 'created': [u'This field is required.']}

In [ ]:
# Return a 400 response if the data was invalid.
serializer.is_valid(raise_exception=True)

# Object-level validation

To do any other validation that requires access to multiple fields, add a method called .validate() to your Serializer subclass. This method takes a single argument, which is a dictionary of field values. It should raise a serializers.ValidationError if necessary, or just return the validated values.

In [ ]:
from rest_framework import serializers

class EventSerializer(serializers.Serializer):
    description = serializers.CharField(max_length=100)
    start = serializers.DateTimeField()
    finish = serializers.DateTimeField()

    def validate(self, data):
        """
        Check that the start is before the stop.
        """
        if data['start'] > data['finish']:
            raise serializers.ValidationError("finish must occur after start")
        return data

# Field-level validation

You can specify custom field-level validation by adding .validate_<field_name> methods to your Serializer subclass. These are similar to the .clean_<field_name> methods on Django forms.

These methods take a single argument, which is the field value that requires validation.

Your validate_<field_name> methods should return the validated value or raise a serializers.ValidationError

In [ ]:
from rest_framework import serializers

class BlogPostSerializer(serializers.Serializer):
    title = serializers.CharField(max_length=100)
    content = serializers.CharField()

    def validate_title(self, value):
        """
        Check that the blog post is about Django.
        """
        if 'django' not in value.lower():
            raise serializers.ValidationError("Blog post is not about Django")
        return value

# Individual fields validation

Individual fields on a serializer can include validators, by declaring them on the field instance, for example:

In [ ]:
def multiple_of_ten(value):
    if value % 10 != 0:
        raise serializers.ValidationError('Not a multiple of ten')

class GameRecord(serializers.Serializer):
    score = IntegerField(validators=[multiple_of_ten])
    ...

Serializer classes can also include reusable validators that are applied to the complete set of field data. These validators are included by declaring them on an inner Meta class, like so:

In [ ]:
class EventSerializer(serializers.Serializer):
    name = serializers.CharField()
    room_number = serializers.IntegerField(choices=[101, 102, 103, 201])
    date = serializers.DateField()

    class Meta:
        # Each room only has one event per day.
        validators = UniqueTogetherValidator(
            queryset=Event.objects.all(),
            fields=['room_number', 'date']
        )

# Dealing with multiple objects

To serialize a queryset or list of objects instead of a single object instance, you should pass the many=True flag when instantiating the serializer. You can then pass a queryset or list of objects to be serialized.

In [ ]:
queryset = Book.objects.all()
serializer = BookSerializer(queryset, many=True)
serializer.data

[

    {'id': 0, 'title': 'The electric kool-aid acid test', 'author': 'Tom Wolfe'},
    {'id': 1, 'title': 'If this is a man', 'author': 'Primo Levi'},
    {'id': 2, 'title': 'The wind-up bird chronicle', 'author': 'Haruki Murakami'}

]

# HyperlinkedModelSerializer

The HyperlinkedModelSerializer class is similar to the ModelSerializer class except that it uses hyperlinks to represent relationships, rather than primary keys.

By default the serializer will include a url field instead of a primary key field.

The url field will be represented using a HyperlinkedIdentityField serializer field, and any relationships on the model will be represented using a HyperlinkedRelatedField serializer field.

You can explicitly include the primary key by adding it to the fields option, for example:


In [ ]:
class AccountSerializer(serializers.HyperlinkedModelSerializer):
    class Meta:
        model = Account
        fields = ('url', 'id', 'account_name', 'users', 'created')

When instantiating a HyperlinkedModelSerializer you must include the current request in the serializer context, for example:

In [ ]:
serializer = AccountSerializer(queryset, context={'request': request})

# ModelSerializer

Often you'll want serializer classes that map closely to Django model definitions.

The ModelSerializer class provides a shortcut that lets you automatically create a Serializer class with fields that correspond to the Model fields.

The ModelSerializer class is the same as a regular Serializer class, except that:

>It will automatically generate a set of fields for you, based on the model.

>It will automatically generate validators for the serializer, such as unique_together validators.

>It includes simple default implementations of .create() and .update().

In [ ]:
class AccountSerializer(serializers.ModelSerializer):
    class Meta:
        model = Account
        fields = ('id', 'account_name', 'users', 'created')

# Specifying which fields to include

In [ ]:
class AccountSerializer(serializers.ModelSerializer):
    class Meta:
        model = Account
        fields = ('id', 'account_name', 'users', 'created')

In [ ]:
class AccountSerializer(serializers.ModelSerializer):
    class Meta:
        model = Account
        fields = '__all__'

In [ ]:
class AccountSerializer(serializers.ModelSerializer):
    class Meta:
        model = Account
        exclude = ('users',)

# Serializer class

1. Create a file in the api directory named serializers.py.
3. A serializer class is very similar to a Django Form class, and includes similar validation flags on the various fields, such as required, max_length and default.
4. The create() and update() methods define how fully fledged instances are created or modified when calling serializer.save()


In [ ]:
from rest_framework import serializers
from my_app import models


class AuthorSerializer(serializers.Serializer):
    id = serializers.IntegerField(read_only=True)
    first_name = serializers.CharField(max_length=100)
    last_name = serializers.CharField(max_length=100)
    date_of_birth = serializers.DateField()
    date_of_death = serializers.DateField('Died',)

    def create(self, validated_data):
        """
        Create and return a new `Author` instance, given the validated data.
        """
        return models.Author.objects.create(**validated_data)

    def update(self, instance, validated_data):
        """
        Update and return an existing `Author`
            instance, given the validated data.
        """
        instance.first_name = validated_data.get(
                                'first_name',
                                instance.first_name
        )

        instance.last_name = validated_data.get(
                                'last_name',
                                instance.last_name
        )

        instance.date_of_birth = validated_data.get(
                                    'date_of_birth',
                                    instance.date_of_birth
        )

        instance.date_of_death = validated_data.get(
                                    'date_of_death',
                                    instance.date_of_death
        )

        instance.save()
        return instance


# Working with Serializers

In [ ]:
python manage.py shell

In [ ]:
>>> from my_app import models
>>> from my_app.api import serializers
>>> from rest_framework.renderers import JSONRenderer

In [ ]:
>>> shakespeare = models.Author(first_name='shakespeare',)
>>> shakespeare.save()
>>> models.Author.objects.all()

<QuerySet [<Author: , shakespeare>]>


In [ ]:
>>> serializers.AuthorSerializer()

AuthorSerializer():
    id = IntegerField(read_only=True)
    first_name = CharField(max_length=100)
    last_name = CharField(max_length=100)
    date_of_birth = DateField()
    date_of_death = DateField('Died')


In [ ]:
>>> serializers.AuthorSerializer(shakespeare)

AuthorSerializer(<Author: , shakespeare>):
    id = IntegerField(read_only=True)
    first_name = CharField(max_length=100)
    last_name = CharField(max_length=100)
    date_of_birth = DateField()
    date_of_death = DateField('Died')


## Python native datatypes

In [ ]:
>>> shakespeare_serialized = serializers.AuthorSerializer(shakespeare)
>>> shakespeare_serialized.data

{'first_name': 'shakespeare', 'last_name': '', 'id': 1, 'date_of_birth': None, 'date_of_death': None}

## rendering Shakespeare into json

In [ ]:
>>> JSONRenderer().render(shakespeare_serialized.data)

b'{"id":2,"first_name":"shakespeare","last_name":"","date_of_birth":null,"date_of_death":null}'

## serialize querysets

In [ ]:
>>> serializer = serializers.AuthorSerializer(models.Author.objects.all(), many=True)
>>> serializer.data

[
    OrderedDict([('id', 3), ('first_name', 'Franz'), ('last_name', 'Kafka'), ('date_of_birth', '2018-03-04'), ('date_of_death', 'Died')])
]



## Using ModelSerializers

### Settings.py

In [ ]:
# Application definition

INSTALLED_APPS = [
    'my_app.apps.MyAppConfig',
    'rest_framework',
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
]

### serailizers.py

In [ ]:
class AuthorSerializer(serializers.ModelSerializer):
    class Meta:
        model = models.Author
        fields = ['id', 'first_name', 'last_name',
                  'date_of_birth', 'date_of_death', ]

### views.py

In [ ]:
from django.http import HttpResponse

from rest_framework import generics

from . import serializers
from my_app import models


def test(request, ):
    return HttpResponse('well done!')


class AuthorListApiView(generics.ListAPIView):
    serializer_class = serializers.AuthorSerializer

    def get_queryset(self, ):
        return models.Author.objects.all()


### url.py

In [ ]:
from django.conf.urls import url

from . import views


urlpatterns = [
    url(r'^test/$', view=views.test, name='test'),
    url(
        r'^author/$',
        view=views.AuthorListApiView.as_view(),
        name='author-list-api-view'
    ),

]

### output

<img src='pics/Screenshot from 2018-03-06 15-21-21.png' />

# Swagger

### settings.py

In [ ]:
INSTALLED_APPS = [
    'my_app.apps.MyAppConfig',
    'rest_framework',
    'rest_framework_swagger',
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
]


### urls.py

In [ ]:
from django.conf.urls import url

from . import views

from rest_framework_swagger.views import get_swagger_view

swagger = get_swagger_view(title='Public API')

urlpatterns = [
    url(r'^test/$', view=views.test, name='test'),
    url(
        r'^author/$',
        view=views.AuthorListApiView.as_view(),
        name='author-list-api-view'
    ),
    url(r'^swagger/$', view=swagger, name='swagger'),
]


<img src='pics/Screenshot from 2018-03-06 15-46-35.png' />